In [1]:
import tensorflow as tf

In [2]:
!nvidia-smi

Wed Jul 20 14:10:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 39%   48C    P0    62W / 260W |    367MiB / 11019MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
#sys.path.insert(0,'/home/ikwak2/hmd/iy_classifier')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

In [4]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [5]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [6]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Jul 20 14:10:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 39%   48C    P0    62W / 260W |    367MiB / 11019MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [8]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'


In [9]:
model_folder = 'lcnn2'

In [10]:
model_folder

'lcnn2'

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [12]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [13]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [14]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 128,
                  'win_length':256,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10
}

In [ ]:
features_trn, mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_rr=True,use_raw=False)

  0%|                                           | 2/751 [00:06<42:48,  3.43s/it]

/home/jk21/Downloads/Data/data/murmur/train/9983_TV.wav


  0%|▏                                          | 3/751 [00:10<42:21,  3.40s/it]/home/jk21/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/.local/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|▎                                          | 6/751 [00:20<42:01,  3.38s/it]

In [ ]:
features_test, _, _, _ = get_features_3lb_all(test_folder, patient_files_test, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_rr=True,use_raw=False)

In [ ]:
pwd

In [ ]:
with open('features_trn.pickle', 'wb') as f:
    pickle.dump(features_trn, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('features_test.pickle', 'wb') as f:
    pickle.dump(features_test, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('features_trn.pickle', 'rb') as f:
    features_trn = pickle.load(f)

In [16]:
with open('features_trn.pickle', 'rb') as f:
    features_test = pickle.load(f)

In [17]:
features_trn.keys()

dict_keys(['id', 'age', 'sex', 'hw', 'preg', 'loc', 'mel1', 'cqt1', 'stft1', 'raw1', 'rr1', 'mm_labels', 'out_labels'])

In [18]:
features_trn['mel1'].shape

(2532, 100, 626, 1)

In [19]:
mel_input_shape = (100,626,1)

In [20]:
cqt_input_shape=(1,1,1)

In [21]:
stft_input_shape=(1,1,1)

In [22]:
model1 = get_LCNN_o_1_dr(mel_input_shape, cqt_input_shape,stft_input_shape,use_mel = True, use_cqt = False, use_stft = False,ext = True,ord1=False)
model2 = get_LCNN_o_1_dr(mel_input_shape, cqt_input_shape,stft_input_shape,use_mel = True, use_cqt = False, use_stft = False,ext = True,ord1=True)


2022-07-20 14:10:27.652072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 14:10:27.656389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 14:10:27.656733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 14:10:27.657620: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [23]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel (InputLayer)               [(None, 100, 626, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 100, 626, 32  832         ['mel[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 100, 626, 32  832         ['mel[0][0]']                    
                                )                                                             

                                                                  'conv2d_15[0][0]']              
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 13, 79, 32)  96          ['maximum_7[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 conv2d_16 (Conv2D)             (None, 13, 79, 32)   1056        ['batch_normalization_5[0][0]']  
                                                                                                  
 conv2d_17 (Conv2D)             (None, 13, 79, 32)   1056        ['batch_normalization_5[0][0]']  
                                                                                                  
 maximum_8 (Maximum)            (None, 13, 79, 32)   0           ['conv2d_16[0][0]',              
          

In [24]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))

batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
                          features_trn['rr1'],features_trn['mel1'],features_trn['stft1'],features_trn['cqt1']], 
                        features_trn['mm_labels'],  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

class_weight = {0: 3, 1: 1., 2: 1} 
    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['rr1'], features_test['mel1'],
                              features_test['stft1'],features_test['cqt1']], 
                             features_test['mm_labels']), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(features_test['mm_labels'])/64),
                           class_weight=class_weight, 
                             epochs = n_epoch)



Epoch 1/100


2022-07-20 14:10:32.738788: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


40/40 [==============================] - 11s 185ms/step - loss: 1.7321 - accuracy: 0.6145 - auc: 0.7964 - val_loss: 7.9217 - val_accuracy: 0.2413 - val_auc: 0.3948 - lr: 9.9851e-04
Epoch 2/100
40/40 [==============================] - 6s 156ms/step - loss: 1.2342 - accuracy: 0.6133 - auc: 0.8100 - val_loss: 3.6468 - val_accuracy: 0.2413 - val_auc: 0.5200 - lr: 9.9831e-04
Epoch 3/100
40/40 [==============================] - 6s 153ms/step - loss: 1.1335 - accuracy: 0.6609 - auc: 0.8358 - val_loss: 0.9872 - val_accuracy: 0.2551 - val_auc: 0.5721 - lr: 9.9807e-04
Epoch 4/100
40/40 [==============================] - 6s 155ms/step - loss: 1.1148 - accuracy: 0.6824 - auc: 0.8407 - val_loss: 0.8735 - val_accuracy: 0.3953 - val_auc: 0.6342 - lr: 9.9781e-04
Epoch 5/100
40/40 [==============================] - 6s 155ms/step - loss: 1.1001 - accuracy: 0.6918 - auc: 0.8496 - val_loss: 0.8124 - val_accuracy: 0.3574 - val_auc: 0.6340 - lr: 9.9750e-04
Epoch 6/100
40/40 [==============================] 

40/40 [==============================] - 6s 157ms/step - loss: 0.9850 - accuracy: 0.7727 - auc: 0.8773 - val_loss: 0.4461 - val_accuracy: 0.8507 - val_auc: 0.9229 - lr: 7.4146e-04
Epoch 44/100
40/40 [==============================] - 6s 158ms/step - loss: 0.9910 - accuracy: 0.7727 - auc: 0.8812 - val_loss: 0.5183 - val_accuracy: 0.8555 - val_auc: 0.9172 - lr: 7.1587e-04
Epoch 45/100
40/40 [==============================] - 6s 157ms/step - loss: 0.9758 - accuracy: 0.7762 - auc: 0.8858 - val_loss: 0.5383 - val_accuracy: 0.8468 - val_auc: 0.9173 - lr: 6.8882e-04
Epoch 46/100
40/40 [==============================] - 6s 157ms/step - loss: 0.9871 - accuracy: 0.7758 - auc: 0.8804 - val_loss: 0.5450 - val_accuracy: 0.8614 - val_auc: 0.9272 - lr: 6.6044e-04
Epoch 47/100
40/40 [==============================] - 6s 158ms/step - loss: 1.0086 - accuracy: 0.7688 - auc: 0.8777 - val_loss: 0.4232 - val_accuracy: 0.8669 - val_auc: 0.9249 - lr: 6.3088e-04
Epoch 48/100
40/40 [============================

40/40 [==============================] - 6s 158ms/step - loss: 0.9820 - accuracy: 0.7805 - auc: 0.8865 - val_loss: 0.4624 - val_accuracy: 0.8910 - val_auc: 0.9329 - lr: 2.1772e-05
Epoch 86/100
40/40 [==============================] - 6s 158ms/step - loss: 0.9482 - accuracy: 0.7875 - auc: 0.8884 - val_loss: 0.4685 - val_accuracy: 0.8918 - val_auc: 0.9316 - lr: 2.0352e-05
Epoch 87/100
40/40 [==============================] - 6s 158ms/step - loss: 0.9976 - accuracy: 0.7801 - auc: 0.8891 - val_loss: 0.4854 - val_accuracy: 0.8851 - val_auc: 0.9281 - lr: 1.9102e-05
Epoch 88/100
40/40 [==============================] - 6s 159ms/step - loss: 0.9749 - accuracy: 0.7824 - auc: 0.8818 - val_loss: 0.4890 - val_accuracy: 0.8823 - val_auc: 0.9268 - lr: 1.8001e-05
Epoch 89/100
40/40 [==============================] - 6s 157ms/step - loss: 0.9879 - accuracy: 0.7688 - auc: 0.8860 - val_loss: 0.4818 - val_accuracy: 0.8847 - val_auc: 0.9293 - lr: 1.7033e-05
Epoch 90/100
40/40 [============================

In [25]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
                          features_trn['rr1'],features_trn['mel1'],features_trn['stft1'],features_trn['cqt1']], 
                        features_trn['out_labels'],  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['rr1'], features_test['mel1'],
                              features_test['stft1'],features_test['cqt1']], 
                             features_test['out_labels']), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(features_test['out_labels'])/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 180ms/step - loss: 0.9491 - accuracy: 0.5027 - auc: 0.5124 - val_loss: 1.0150 - val_accuracy: 0.5213 - val_auc: 0.5670 - lr: 9.9851e-04
Epoch 2/100
40/40 [==============================] - 6s 159ms/step - loss: 0.7283 - accuracy: 0.5336 - auc: 0.5363 - val_loss: 0.7188 - val_accuracy: 0.5225 - val_auc: 0.5726 - lr: 9.9831e-04
Epoch 3/100
40/40 [==============================] - 6s 159ms/step - loss: 0.7133 - accuracy: 0.5273 - auc: 0.5395 - val_loss: 0.6839 - val_accuracy: 0.5596 - val_auc: 0.5836 - lr: 9.9807e-04
Epoch 4/100
40/40 [==============================] - 6s 158ms/step - loss: 0.7043 - accuracy: 0.5359 - auc: 0.5475 - val_loss: 0.7018 - val_accuracy: 0.5371 - val_auc: 0.5635 - lr: 9.9781e-04
Epoch 5/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6953 - accuracy: 0.5336 - auc: 0.5397 - val_loss: 0.6794 - val_accuracy: 0.5707 - val_auc: 0.5977 - lr: 9.9750e-04
Epoch 6/100
40/40 [=====================

40/40 [==============================] - 6s 158ms/step - loss: 0.6772 - accuracy: 0.5684 - auc: 0.6422 - val_loss: 0.6412 - val_accuracy: 0.6465 - val_auc: 0.7084 - lr: 7.4146e-04
Epoch 44/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6745 - accuracy: 0.5852 - auc: 0.6562 - val_loss: 0.6482 - val_accuracy: 0.5900 - val_auc: 0.6454 - lr: 7.1587e-04
Epoch 45/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6736 - accuracy: 0.5844 - auc: 0.6438 - val_loss: 0.6830 - val_accuracy: 0.5825 - val_auc: 0.6025 - lr: 6.8882e-04
Epoch 46/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6777 - accuracy: 0.5746 - auc: 0.6327 - val_loss: 0.6349 - val_accuracy: 0.6493 - val_auc: 0.7037 - lr: 6.6044e-04
Epoch 47/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6762 - accuracy: 0.5836 - auc: 0.6462 - val_loss: 0.6346 - val_accuracy: 0.6536 - val_auc: 0.7106 - lr: 6.3088e-04
Epoch 48/100
40/40 [============================

40/40 [==============================] - 6s 158ms/step - loss: 0.6670 - accuracy: 0.5996 - auc: 0.6634 - val_loss: 0.6230 - val_accuracy: 0.6758 - val_auc: 0.7316 - lr: 2.1772e-05
Epoch 86/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6618 - accuracy: 0.6000 - auc: 0.6862 - val_loss: 0.6195 - val_accuracy: 0.6742 - val_auc: 0.7383 - lr: 2.0352e-05
Epoch 87/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6677 - accuracy: 0.5934 - auc: 0.6833 - val_loss: 0.6190 - val_accuracy: 0.6718 - val_auc: 0.7432 - lr: 1.9102e-05
Epoch 88/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6699 - accuracy: 0.5914 - auc: 0.6648 - val_loss: 0.6189 - val_accuracy: 0.6730 - val_auc: 0.7420 - lr: 1.8001e-05
Epoch 89/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6653 - accuracy: 0.6043 - auc: 0.6795 - val_loss: 0.6190 - val_accuracy: 0.6671 - val_auc: 0.7420 - lr: 1.7033e-05
Epoch 90/100
40/40 [============================

In [26]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 128,
 'win_length': 256,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10}

In [27]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [28]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 128,
 'win_length': 256,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 626, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1)}

In [29]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [30]:
model_folder

'lcnn2'

In [31]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1', m_name2 = 'lcnn2', param_feature = params_feature)

1

In [32]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [33]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 128,
 'win_length': 256,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 626, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [96]:
def run_challenge_model(model, data, recordings, verbose,use_mel=True,use_stft=False,use_cqt=False):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy1' :
        model1 = get_toy5_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'] )
    if model['model2'] == 'toy2' :
        model2 = get_toy5_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'])
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_o_1_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False,ord1=False,ext=True)
    if model['model2'] == 'lcnn2' :
        model2 = get_LCNN_o_1_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False,ord1=True,ext=True)
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['rr1'] = []
    
    for i in range(len(recordings)):
        try:
#             ____, recording1 = sp.io.wavfile.read(recordings)
            ____, info = nk.ecg_process(recordings[i], sampling_rate=4000)
            rr = np.mean(np.diff(info['ECG_R_Peaks'])/4000)
            if rr>1:
                current_rr = np.array([0,0,1])
            elif rr<0.6:
                current_rr = np.array([1,0,0])
            else :
                current_rr = np.array([0,1,0])
        except:
            current_rr=np.array([0,0,0])
        
        features['rr1'].append(current_rr)  
    
    features['rr1'] = np.array(features['rr1'])
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        if use_mel :
            mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        else :
            mel1 = np.zeros( (1,1) )
            
        features['mel1'].append(mel1)
        
    M, N = features['mel1'][0].shape
    
    if use_mel :
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    
    features['mel1'] = np.array(features['mel1'])

  
    features['cqt1'] = []
    for i in range(len(recordings)) :
        if use_cqt :
            mel1 = feature_extract_cqt(recordings[i], samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        else:
            mel1 = np.zeros( (1,1) )
            
        features['cqt1'].append(mel1)
        
    M, N = features['cqt1'][0].shape
    
    if use_cqt :
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    
    features['cqt1'] = np.array(features['cqt1'])
    
    
    
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        if use_stft :
            mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                        win_length = win_length)[0]
        else :
            mel1 = np.zeros( (1,1) )
        
        features['stft1'].append(mel1)
        
    M, N = features['stft1'][0].shape
    if use_stft :
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['rr1'],features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['rr1'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

In [97]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [98]:
output_folder = '/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [99]:
model_folder

'lcnn2'

In [107]:
test_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [108]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


/home/jk21/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/.local/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


    2/191...
    3/191...


/home/jk21/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/.local/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


    4/191...
    5/191...


...

    188/191...


/home/jk21/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/.local/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


    189/191...
    190/191...


/home/jk21/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jk21/.local/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


    191/191...
Done.


In [104]:
from evaluate_model import *

In [105]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.884,0.693,0.495,0.759,0.757,15329.847

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.753,0.741,0.697,0.702,0.607,14282.727

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.942,0.820,0.891
AUPRC,0.858,0.278,0.945
F-measure,0.636,0.000,0.851
Accuracy,0.895,0.000,0.799

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.753,0.753
AUPRC,0.805,0.677
F-measure,0.659,0.735
Accuracy,0.561,0.849

